In [1]:
#from transformer import Transformer 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
import os
import gc
import random
import numpy as np
#import polars as pl
import pandas as pd
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
import fastai
from fastai.vision.all import *

import math
import copy

# Check if a GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device = "cpu"


max_seq_len = 510 + 2  # +2 is added to cover start and end token of sequence This will change to 457 in private set  # Set the maximum sequence length to 100
data_preprocessing_done = 0
save_pkl = 0
load_pkl = 0
small_dataset = 1

#Training Data
if small_dataset == 1:
    
    #Uncomment this section when uploading the jupyter notebook
    '''
    df_read = pd.read_csv('/media/spartans/COMMON:/RNA_Project/train_data.csv')
    
    criteria_2A3 = (df_read['experiment_type'] == '2A3_MaP') & (df_read['SN_filter'] == 1)
    criteria_DMS = (df_read['experiment_type'] == 'DMS_MaP') & (df_read['SN_filter'] == 1)

    df_2A3_Filter1 = df_read[criteria_2A3]
    df_2A3_Filter1 = df_2A3_Filter1.head(4)

    df_DMS_Filter1 = df_read[criteria_DMS]
    df_DMS_Filter1 = df_DMS_Filter1.head(4)

   
    df = pd.concat([df_2A3_Filter1, df_DMS_Filter1], ignore_index=True)
    
    df.to_csv('2A3_DMS_With_Filter1_Train_Data.csv', index=False)
    #df_2A3_Filter1.to_csv('2A3_DMS_With_Filter1_Train_Data.csv', index=False)
    '''
    df = pd.read_csv('2A3_DMS_With_Filter1_Train_Data.csv')
    #df = pd.read_csv('DMS_Data.csv')
    #df = pd.read_csv('2A3_Data.csv')
else:
    df = pd.read_csv('/media/spartans/COMMON:/RNA_Project/train_data.csv')
    

#test_df = pd.read_csv('/media/spartans/COMMON:/RNA_Project/test_sequences.csv')
TEST_PATH = "/media/spartans/COMMON:/RNA_Project/test_sequences.csv"
SUBMISSION_PATH = "/media/spartans/COMMON:/RNA_Project/submission_0.csv"
SAMPLE_SUBMISSION_PATH = "/media/spartans/COMMON:/RNA_Project/sample_submission.csv"
MODEL_NAME = "Trained_Model.model_512_11_18_2023"

seed=69
os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)

os.environ['TORCH_USE_CUDA_DSA'] = str(1)


def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(seed)

In [2]:



#criteria_2A3 = (df_read['experiment_type'] == '2A3_MaP') & (df_read['SN_filter'] == '1')
#criteria_DMS = (df_read['experiment_type'] == 'DMS_MaP') & (df_read['SN_filter'] == '1')

#criteria_2A3 = ((df_read['experiment_type'] == 'DMS_MaP')  & df_read['SN_filter'] == 1 )
#df_2A3_Filter1 = df_read[criteria_2A3]
#df_2A3_Filter1
#df

In [3]:
#where reactivity is NaN,
# have Nan replaced with average of all numbers in the sequence.

#Map sequence letters to numbers.

#Looking at Reads and Signal To Noise ..they appear to be somewhat coorelated.

#Create Dataset function


#In contrastive loss model... you can train based on#
#in a sequence which part is 2D fold and which part is 3D fold
#See notebook https://www.kaggle.com/code/something4kag/ribonanza-3d-coords-prep
#if it can be helpful
# Also analyze and which position in the sequence the fold can occur and which fold ended up happening based on the data

#another method to formulate contrastive learning model is by deferentiating sequences with low SNR and hence reactivity value will not be with high confidence.



In [4]:

#In RNA, the most common base pairings you'll find are between the following nucleotide bases:

#Adenine (A) and Uracil (U): A forms base pairs with U. This is a fundamental pairing and is commonly seen in RNA molecules, particularly in single-stranded regions. It is important for the stability of stem-loop structures and is a key component of RNA secondary structure.

#Guanine (G) and Cytosine (C): G forms base pairs with C, just as it does in DNA. This pairing is less common in RNA secondary structure but is still important for certain RNA molecules. It may be more prevalent in the context of ribozymes and catalytic RNA.

#While A-U and G-C are the primary base pairings in RNA, it's essential to understand that RNA can also exhibit non-canonical or non-standard base pairings, especially in more complex RNA structures. These non-canonical pairings can involve different combinations of A, U, G, and C and are often seen in tertiary structures or specialized RNA molecules with specific functions.

#The prevalence of A-U and G-C base pairings in an RNA molecule can vary depending on its sequence and function. For instance, regions that need to form stable secondary structures often rely on A-U pairings, while regions involved in catalytic activities may include G-C pairs. RNA structures are diverse and can exhibit a wide range of base pairing interactions to achieve their biological functions.



#So a contrastive loss can be formed here where
#AU pair GC pair are strong positive - stable
#AG, UC are less stable
#UG, UA, CG pair are unstable so negative .
#
#In RNA, the stable base pairs among the four nucleotide bases (A, G, U, and C) follow the standard Watson-Crick base-pairing rules. Here are the stable base pairs among these bases:

#GC pairs are more stable than AU pair


#So you can tokenzie these pairs in the transformer model.

In [5]:
#Formulate Convolution NN or Dense Net which takes SNR, Reactivity error and feeds into the final layer of Transformer network
# As an example
#Lowest Reactivity Error and High SNR are  strong positives
# Lowest SNR are strong negative.





#sequence length range from 115 to 206 in train dataset.
#in final it will range from 207 to 457

In [6]:
#In positional encoding
#pos means the position of the word in the sequence.
#i means the index for that particular word vector (index of the dimension)
#dmodel is the dimension for e.g 512 from the example


#For 2 experiment types i.e. DMS and 2A3 you can have start and end  token in a sequence for which data is available.
#.e. start_DMS, end_DMS, start_2A3, end_2A3.




In [7]:
if data_preprocessing_done == 0:
    #Assigning reactivity Columns to react_columns dataframe and replacing NaN with 0.000 

    react_columns = df.iloc[:,7:213].fillna(0.000000)

    #Find mean of reactivity per row
    react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)

    react_columns_mean = react_columns.mean(axis=1)
    #Free up memory
    del(react_columns)

    #Now replace NaN in react_columns with mean_reactivity
    #react_columns_New = df.iloc[:,7:213].fillna(mean_reactivity)

/tmp/ipykernel_13228/3653553058.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)


In [8]:

if data_preprocessing_done == 0:
    react_columns = df.iloc[:,7:213] #You may have to capture those indices as per sequence length i.e. when it extends to 457

    # Replace NaN values in each row with values from replace_values array
    for index, row in tqdm(react_columns.iterrows(), total=len(react_columns), desc="Processing rows"):
        react_columns.loc[index] = row.fillna(react_columns_mean[index])
    
    react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)


        
        
        

Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 5188.56it/s]
/tmp/ipykernel_13228/1102462686.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  react_columns = react_columns.applymap(lambda x: 0 if isinstance(x, (float, int)) and x < 0 else x)


In [9]:
seq_map = {'A':1.0,
           'C':2.0,
           'G':3.0,
           'U':4.0, 
           'M':5.0,  #Start token of DMS exp seq
           'N':6.0,  #End token of DMS exp seq
           'T':7.0,  #Start token of 2A3 exp seq
           'X':8.0,  #End token of 2A3 exp seq
           'Z':0.0   #Padding token
          }


In [10]:
# Iterate over rows and replace a specific value in all columns


#max_seq_len = 206 #This will change to 457 in private set  # Set the maximum sequence length to 100
if data_preprocessing_done == 0:
    df_sq = copy.deepcopy(df['sequence']) #saving a backup of sequences.
    #react_columns['maped_sequence'] = [' ']*len(react_columns)
    #for index, row in tqdm(df.iterrows(), total=len(df.rows), desc="Processing rows"):
    for index, row in tqdm(react_columns.iterrows(), total=len(react_columns), desc="Processing rows"):
        if df.at[index, 'experiment_type'] == "2A3_MaP":
            concat_seq = 'T' + df.at[index, 'sequence'] + 'X'
        if df.at[index, 'experiment_type'] == "DMS_MaP":
            concat_seq = 'M' + df.at[index, 'sequence'] + 'N'
        padding = "Z"*(max_seq_len - 2 - len(df.at[index, 'sequence']))
        concatenated_seq = concat_seq + padding
        df.at[index, 'sequence'] = [seq_map[s] for s in concatenated_seq]
        #react_columns.at[index, 'maped_sequence'] = [seq_map[s] for s in concatenated_seq]


Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 7484.82it/s]


In [11]:

if data_preprocessing_done == 0:
    #react_columns['reactivity_0207'] = copy.deepcopy(react_columns['reactivity_0206'])
    #react_columns['reactivity_0208'] = copy.deepcopy(react_columns['reactivity_0206'])
    for iii in range(207,511):
        react_col_name = "reactivity_0" + str(iii)
        react_columns[react_col_name] = copy.deepcopy(react_columns['reactivity_0206'])
        
    #replicated_columns = [react_columns['reactivity_0206'].repeat(2)]
    #react_columns = pd.concat([react_columns, replicated_columns] , axis=1, ignore_index=True)
    #print(react_columns)
    react_columns['mapped_sequence'] = copy.deepcopy(df['sequence'])
    #del react_columns['reactivity_0207']
    #del react_columns['reactivity_0208']
if save_pkl == 1:
    react_columns.to_pickle("react_columns.pkl")

if load_pkl == 1:
    react_columns = pd.read_pickle("react_columns.pkl")
    
#react_columns.columns

/tmp/ipykernel_13228/834134542.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  react_columns[react_col_name] = copy.deepcopy(react_columns['reactivity_0206'])
/tmp/ipykernel_13228/834134542.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  react_columns[react_col_name] = copy.deepcopy(react_columns['reactivity_0206'])
/tmp/ipykernel_13228/834134542.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

In [12]:
# Define a custom dataset class for your data
class CustomDataset(Dataset):
    def __init__(self, data, max_seq_len):
        self.data = data
        self.max_seq_len = max_seq_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        trg_sequence = torch.tensor(self.data.iloc[idx, :].tolist()[:-1])
        src_sequence = torch.tensor(self.data.iloc[idx, :].tolist()[-1])
        
        #print("Size src seq ", src_sequence.size())
        #print("Size trg seq ", trg_sequence, trg_sequence.size())
        
        return src_sequence, trg_sequence

    
# Create a DataLoader for batching and shuffling
batch_size = 64
#batch_size = 3


# Define the size of the training set
dataset_size = len(react_columns)
train_size = int(0.9 * dataset_size)
test_size = dataset_size - train_size

custom_dataset = CustomDataset(react_columns, max_seq_len)
train_indices, test_indices = random_split(custom_dataset, [train_size, test_size])
train_dataloader = DataLoader(train_indices, batch_size=batch_size)
test_dataloader = DataLoader(test_indices, batch_size=batch_size)


    

In [13]:

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)
        
    def forward(self, x):
        self.encoding = self.encoding.to(x.device)
        return x + self.encoding[:, :x.size(1)].detach()
    


class MyTransformerEncoderLayer(nn.TransformerEncoderLayer):
    def __init__(self, d_model, nhead, batch_first=True, **kwargs):
        super(MyTransformerEncoderLayer, self).__init__(d_model, nhead, **kwargs)
        self.self_attn = nn.MultiheadAttention(d_model, nhead, batch_first=batch_first)


class TransformerRegressionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, nhead, num_layers, output_sequence_length, max_len=512):
        super(TransformerRegressionModel, self).__init__()
        
        self.embedding = nn.Embedding(input_dim, hidden_dim)
        
        self.positional_encoding = PositionalEncoding(hidden_dim, max_len=max_len)
        
        
        '''
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=nhead),
            num_layers=num_layers
        )
        '''
        self.transformer_encoder = nn.TransformerEncoder(
            MyTransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, batch_first=True),
            num_layers=num_layers
        )
        
        
        
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=input_dim, nhead=nhead),
            num_layers=num_layers
        )
        
        
        
        
        self.fc = nn.Linear(input_dim, output_sequence_length * 1)
        self.gelu = nn.GELU()
        self.output_sequence_length = output_sequence_length


    
    def forward(self, x):              
        
        # Ensure that both the model and input are on the same device
        device = next(self.parameters()).device
        x = x.to(device)
        
        # Apply embedding to input
        #print("X shape 1", x.shape)
        x = self.embedding(x)
        #print("X shape 2", x.shape)
        
        x = self.positional_encoding(x)
        
        # Permute to (sequence_length, batch_size, input_dim)
        x = x.permute(1, 0, 2)
        #print("X shape 3", x.shape)
        
        # Encoder forward pass
        encoder_output = self.transformer_encoder(x)

        # Decoder forward pass with memory from the encoder
        decoder_output = self.transformer_decoder(x, memory=encoder_output)

        # Global average pooling over the sequence dimension
        x = decoder_output.mean(dim=0)

        # Final fully connected layer with GELU activation
        x = self.gelu(self.fc(x))

        return x
    

# Hyperparameters
input_dim =  max_seq_len # Adjust based on your input size
hidden_dim = 512
nhead = 16
num_layers = 6
output_sequence_length = max_seq_len - 2  # Adjust based on your output sequence length
#batch_size = 1
num_epochs = 10
learning_rate = 0.001


In [ ]:
print(device)

# Instantiate the model, loss function, and optimizer
model = TransformerRegressionModel(input_dim, hidden_dim, nhead, num_layers, output_sequence_length).to(device)
#criterion = nn.MSELoss()
criterion = nn.L1Loss(size_average=None, reduce=None, reduction='mean')
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)

optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9, weight_decay=0, amsgrad=False)



training_losses = []  # to store training losses
validation_losses = []
prev_average_val_loss = 10000000000
list_of_avg_loss = []
epochss = []
# Training loop
for epoch in range(num_epochs):
    epoch_loss = 0.0  # accumulate loss for the entire epoch
    val_loss = 0.0
    #for inputs, targets in tqdm(train_dataloader.iterrows(), total=len(train_dataloader), desc="Processing rows"):
    #with tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch') as tqdm_loader:
    #    for inputs, targets in tqdm_loader:
         
    model.train()        
    #for batch, dat in enumerate(train_dataloader, 1): 
    for batch, dat in enumerate(tqdm(train_dataloader, desc="Processing batches", unit="batch"), 1):
            inputs = dat[0].to(device, dtype=torch.long)
            targets = dat[1].to(device).float()
            #print("INPUT ", inputs)
            #print("TARGET ", targets)
            optimizer.zero_grad()
            outputs = model(inputs)
            #print("OUT", outputs)
            #print(f'TRAIN:  Batch {batch}, Input: {inputs.shape}, Outputs: {outputs.shape}, Targets: {targets.shape}')
            #Pre-clipping may mean the model may not learn the targets.
            outputs[outputs < 0] = 0.0
            outputs[outputs > 1] = 1.0
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            #print(f"Epoch: {epoch+1}, Training Loss: {loss.item()}")
    
    model.eval()
    with torch.no_grad():
        #for batch, dat in enumerate(test_dataloader, 1):
        for batch, dat in enumerate(tqdm(test_dataloader, desc="Processing batches", unit="batch"), 1):
            inputs = dat[0].to(device, dtype=torch.long)
            targets = dat[1].to(device).float()
            output = model(inputs)
            #print(f'VAL:  Batch {batch}, Input: {inputs.shape}, Outputs: {output.shape}, Targets: {targets.shape}')
            loss = criterion(output, targets)
            #print(f"Epoch: {epoch+1}, Validation Loss: {loss.item()}")
            val_loss += loss.item()
    
    #outputs.requires_grad_(True)
    
    
    average_epoch_loss = epoch_loss / len(train_dataloader)
    training_losses.append(average_epoch_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {average_epoch_loss:.4f}')
    
    average_val_loss = val_loss / len(test_dataloader)
    validation_losses.append(average_val_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {average_val_loss:.4f}')
    epochss.append(epoch)
    
    list_of_avg_loss.append(average_val_loss)
    #Save best model Every Epoch
    if average_val_loss < prev_average_val_loss:
        torch.save(model.state_dict(), MODEL_NAME)
        
        
        
        print("Model Saved because ")
        print(f'Current average validation loss {average_val_loss:.4f} is less than previous average validation loss {prev_average_val_loss:.4f}')
    prev_average_val_loss = min(list_of_avg_loss)


# Plotting the training loss
plt.plot(epochss, training_losses, label='Training Loss')
plt.plot(epochss, validation_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()
# After training, you can use the model for prediction
# Example usage:
# test_input = torch.randn(1, sequence_length, input_dim)  # Adjust based on your input size and sequence length
# predicted_output = model(test_input)


In [14]:
#Validation predict
#val_df = 

#Read TEST Sequence
test_df = pd.read_csv(TEST_PATH)

In [15]:
test_df.head(4)
test_df.at[0, 'sequence']

'GGGAACGACUCGAGUAGAGUCGAAAAUUUCCUUCCAAAUCCUGAGGGAGAGAUAGAGGCGGAGGGUCUGGGGGAGGAAUUAAAACACAAGGUCUCCUCCCCUCUCGCCUGUCCGAACUUGGGGGCACCCCGGCUCGUACUUCGGUACGAGCCGGGGAAAAGAAACAACAACAACAAC'

In [16]:
len(test_df['sequence'][0])

177

In [17]:
test_df['DMS_sequence'] = copy.deepcopy(test_df['sequence'])
test_df['2A3_sequence'] = copy.deepcopy(test_df['sequence'])


test_df_sq = copy.deepcopy(test_df['sequence']) #saving a backup of sequences.
#react_columns['maped_sequence'] = [' ']*len(react_columns)
#for index, row in tqdm(df.iterrows(), total=len(df.rows), desc="Processing rows"):
for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing rows"):
    
    padding = "Z"*(max_seq_len - 2 - len(test_df.at[index, 'sequence']))
    form_DMS_seq = 'M' + test_df.at[index, 'sequence'] + 'N' + padding
    form_2A3_seq = 'T' + test_df.at[index, 'sequence'] + 'X' + padding
    

    test_df.at[index, 'DMS_sequence'] = [seq_map[s] for s in form_DMS_seq]
    test_df.at[index, '2A3_sequence'] = [seq_map[s] for s in form_2A3_seq]
    #if small_dataset == 1:
    #    break
    

Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████████| 1343823/1343823 [03:12<00:00, 6993.25it/s]


In [11]:
#torch.no_grad():
#src_data = src_data.to(device=device)
#tgt_data = tgt_data.to(device=device, dtype=torch.long)
#output = transformer_model(src_data, tgt_data[:, :-1])

#out = transformer_model(test_df['2A3_sequence'][0], tgt_data[:, :-1])

input_sequence = torch.tensor([test_df.at[357440, 'DMS_sequence']]).to(device=device, dtype=torch.long)
#inputs = dat[0].to(device, dtype=torch.long)
out = []
#print("input seq", input_sequence.size())
out = model(input_sequence)
#print("out", out.size, out.shape)
out[out < 0] = 0.0
out[out > 1] = 1.0
#out
index = 357440
seq_length = len(test_df.at[index, 'sequence'])
padding = "Z"*(max_seq_len - 2 - len(test_df.at[357440, 'sequence']))
id_minn = test_df.at[index, 'id_min']
id_maxx = test_df.at[index, 'id_max']
#print(f'index {index}, seq len {seq_length}, id_min {id_minn},  id_max {id_maxx} , difference {id_maxx - id_minn}')
#print(test_df.at[index, 'DMS_sequence'])
#print(f'pad -{padding}-')

In [21]:
#model  = torch.load(MODEL_NAME)
model = TransformerRegressionModel(input_dim, hidden_dim, nhead, num_layers, output_sequence_length).to(device)
model.load_state_dict(torch.load('With_State_DicT'))

model.eval()

TransformerRegressionModel(
  (embedding): Embedding(512, 512)
  (positional_encoding): PositionalEncoding()
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x MyTransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_p

In [19]:
#sample_df = pd.read_csv("/media/spartans/COMMON:/RNA_Project/sample_submission.csv")
sample_df = pd.read_csv(SAMPLE_SUBMISSION_PATH)



In [22]:
test_df['id_max'][0]
#print(test_df)
sample_df['reactivity_DMS_MaP'] = sample_df['reactivity_DMS_MaP'].astype(float)
sample_df['reactivity_2A3_MaP'] = sample_df['reactivity_DMS_MaP'].astype(float)

for index, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Processing rows"):
    input_sequence = torch.tensor([test_df.at[index, 'DMS_sequence']]).to(device=device, dtype=torch.long)
    seq_length = len(test_df.at[index, 'sequence'])
    #print(f'{seq_length}')
    #if seq_length != 177:
        #print(f' seq length {seq_len}')
        #id_minn = test_df.at[index, 'id_min']
        #id_maxx = test_df.at[index, 'id_max']
        #print(f'index {index}, seq len {seq_length}, id_min {id_minn},  id_max {id_maxx} ')
    #First check what is the sequence length. If it is less than or equal to 206 then process it straight.
    
    id_minn = test_df.at[index, 'id_min']
    id_maxx = test_df.at[index, 'id_max']
    #print(f'index {index}, seq len {seq_length}, id_min {id_minn},  id_max {id_maxx} ')
    
    #out_dms = []
    out_dms = model(input_sequence)
    out_dms[out_dms < 0] = 0.0
    out_dms[out_dms > 1] = 1.0
    
    out_dms = out_dms.tolist()   
    iiii = 0
    for iii in range(id_minn, id_maxx): 
        sample_df.at[iii, 'reactivity_DMS_MaP'] = out_dms[0][iiii]
        sample_df.at[iii, 'id'] = iii
        iiii = iiii + 1


    #out_2a3 = []
    input_sequence = torch.tensor([test_df.at[index, '2A3_sequence']]).to(device=device, dtype=torch.long)
    out_2a3 = model(input_sequence)
    out_2a3[out_2a3 < 0] = 0.0
    out_2a3[out_2a3 > 1] = 1.0
    out_2a3 = out_2a3.tolist()
    #out_2a3[:seq_length]
    
    #sample_df[id_minn:id_maxx]['reactivity_2A3_MaP'] = pd.DataFrame({'reactivity_2A3_MaP': out_2a3[0][:seq_length]}, index=range(id_minn, id_minn + seq_length))
    iiii = 0 
    for iii in range(id_minn, id_maxx):
        sample_df.at[iii, 'reactivity_2A3_MaP'] = out_2a3[0][iiii]    
        iiii = iiii + 1  

sample_df['id'] = sample_df['id'].astype(int)
columns_order = ['id','reactivity_DMS_MaP','reactivity_2A3_MaP']
out_sample_df = sample_df[columns_order]
out_sample_df.to_csv(SUBMISSION_PATH, index=False, float_format='%.6f')
    #if index==2:
    #    break
    
    
    
#sample_df   

Processing rows: 100%|████████████████████████████████████████████████████████████████████████████████| 1343823/1343823 [15:14:39<00:00, 24.49it/s]
